In [112]:
from IPython.display import display
pd.options.display.max_rows = None

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [7]:
BASE_URL = 'http://web.mta.info/developers/turnstile.html'
r = requests.get(BASE_URL)

uri_list = ["http://web.mta.info/developers/" + uri[6:] for uri in re.findall(r'href="data/nyct/turnstile/turnstile_1[34567]\d{4}.txt',r.text)]
uri_list[0],uri_list[-1]

('http://web.mta.info/developers/data/nyct/turnstile/turnstile_171230.txt',
 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_130105.txt')

In [213]:
data = pd.read_csv(uri_list[0])
data

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,03:00:00,REGULAR,6455840,2184987
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,07:00:00,REGULAR,6455856,2184995
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,11:00:00,REGULAR,6455899,2185082
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,15:00:00,REGULAR,6456038,2185156
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/23/2017,19:00:00,REGULAR,6456327,2185197
...,...,...,...,...,...,...,...,...,...,...,...
197210,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,12/29/2017,04:00:00,REGULAR,2843616,384153
197211,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,12/29/2017,08:00:00,REGULAR,2843681,384156
197212,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,12/29/2017,12:00:00,REGULAR,2844122,384160
197213,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,12/29/2017,16:00:00,RECOVR AUD,2844899,384168


In [217]:
new_cols = [column.strip() for column in data.columns]
data.rename(columns=dict(zip(data.columns,new_cols)), inplace=True)
data['DATE'] = pd.to_datetime(data['DATE'])
data2017 = data[data['DATE'] > '2017-01-01']
data2017.sort_values(by='UNIT,SCP,DATE,TIME'.split(','), inplace=True)

In [218]:
data2017 = data2017[data2017.groupby(['UNIT','SCP'])['ENTRIES'].transform(max)  == data2017['ENTRIES']]['C/A,UNIT,SCP,ENTRIES,EXITS'.split(',')]
data2017 = data2017.drop_duplicates(keep='last') # Last ocurrence is older
data2017

,C/A,UNIT,SCP,ENTRIES,EXITS
10047,A060,R001,00-00-00,2975207,2144649
10089,A060,R001,00-00-01,2477069,1694184
10131,A060,R001,00-00-02,60596,48501
10173,A060,R001,00-00-03,2123337,104601
10215,A060,R001,00-00-04,188237,202571
...,...,...,...,...,...
109912,N702A,R572,01-03-00,89149,157513
109954,N702A,R572,01-03-01,106933,148205
109996,N702A,R572,01-03-02,160746,164353
110039,N702A,R572,01-03-03,281189,190938


In [219]:
data2017['ENTRIES'] = data2017.groupby(['UNIT'])['ENTRIES'].transform(sum)
data2017['EXITS'] = data2017.groupby(['UNIT'])['EXITS'].transform(sum)
data2017 = data2017['UNIT,ENTRIES,EXITS'.split(',')].drop_duplicates(subset=['UNIT'])
data2017

,UNIT,ENTRIES,EXITS
10047,R001,303751604,88597647
37870,R003,6527311,3863719
37996,R004,11473263,10044936
38206,R005,19269868,16082447
38373,R006,54959266,15295439
...,...,...,...
112230,R551,5058938,5418544
111360,R552,8207887,85676075
108110,R570,125455733,7060067
108812,R571,43915742,55668573


In [232]:
for uri in uri_list:
    if '_1612' in uri:
        print(uri)

http://web.mta.info/developers/data/nyct/turnstile/turnstile_161231.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_161224.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_161217.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_161210.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_161203.txt


In [270]:
URI = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_161231.txt'
data = pd.read_csv(URI)

In [277]:
data['DATE'] = pd.to_datetime(data['DATE'])

data.sort_values(by='UNIT,SCP,DATE,TIME'.split(','), inplace=True)

data24 = data[(data['DATE'] == '2016-12-24')].copy()

In [278]:
new_cols = [column.strip() for column in data24.columns]
data24.rename(columns=dict(zip(data24.columns,new_cols)), inplace=True)

In [288]:
data24['E_'] = data24.groupby(['UNIT','SCP'])['ENTRIES'].transform(max) - data24.groupby(['UNIT','SCP'])['ENTRIES'].transform(min)
data24['S_'] = data24.groupby(['UNIT','SCP'])['EXITS'].transform(max) - data24.groupby(['UNIT','SCP'])['EXITS'].transform(min)
data24['TOTAL'] = data24['E_'] + data24['S_']

In [294]:
result = data24['UNIT,TOTAL'.split(',')].sort_values(['TOTAL'], ascending=False).drop_duplicates()
result.to_csv('result_stations.csv', index=False)